# Learning Langchain (The book)
___

The main task of the software engineer working with LLMs is not to train an LLM, or even to fine-tune one (usually), but rather to take an existing LLM and work out how to get it to accomplish the task you need for your application. Adapting an existing LLM for your task is called `prompt engineering`

`prompt engineering with LangChain` - how to use LangChain to get LLMs to do what do what you have in mind.

## Prompt technique
___

* Zero-Shot Prompting
* Chain-of-Thought(CoT)
* Retrieval-Augmented Generation - in real applications should be combined with CoT
* Tool Calling - consist of prepending the prompt with a list of external functions the LLM can make use of, along with descriptions of what is good for nd instructions on how to use one (or more) of these functions.The developer of the application - should parse the output and call the appropriate functions.
* Few-Shot Prompting

Important things to keep in mind when prompting LLMs: each prompting technique is most useful when used in combination with (some of) the others.

## Retrieval-Augmented Generation (RAG)

**RAG** é uma técnica que combina **recuperação de informações** com **geração de texto**. Em vez de depender apenas do conhecimento pré-treinado do LLM, o RAG busca informações relevantes em uma base de dados externa e usa essas informações como contexto para gerar respostas mais precisas e atualizadas.

### Funcionamento do RAG:
1. **Indexação**: Documentos são vetorizados e armazenados
2. **Recuperação**: Query do usuário busca documentos similares
3. **Geração**: LLM usa documentos recuperados como contexto

`LLM interface simply takes a string prompt as input, sends the input to the model provider, and then returns the model prediction as output.`

## Capítulo 1
___

a - Chamada a um llm

In [35]:
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI  # type: ignore
from langchain_core.prompts import ChatPromptTemplate  # type: ignore
import os

#* Carrega as variáveis de ambiente
_ = load_dotenv(find_dotenv())

#* Verifica se a API key está configurada
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY não encontrada no arquivo .env")

#* Configura o modelo com parâmetros específicos
model: ChatOpenAI = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.0,  # Controla a criatividade das respostas
)  # type: ignore

response = model.invoke("The Sky is ?")
print(response.content)

blue


b - Chatmodel

`HumanMessage`

In [36]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage

model = ChatOpenAI()
prompt = [HumanMessage("What is the capital of Brasil?")]
response = model.invoke(prompt)
print(response.content)

The capital of Brazil is Brasília.


c - System

`SystemMessage`

In [37]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

model = ChatOpenAI()
system_msg = """You are a helpful assistant that responds to questions with three exclamations marks.
"""
human_msg = HumanMessage('What is the capital of France?')
response = model.invoke([system_msg, human_msg])
print(response.content)

Paris!!!


Prompt instructions significantly influences the model's output. Prompts help the model understand context and generate relevant answers to queries.

LangChain provides prompt template interfaces that make it easy to construct prompts with dynamic inputs

In [38]:
from langchain_core.prompts import PromptTemplate

template = PromptTemplate.from_template("""Answer the questions based on the context below.
If the question cannot be answered using the information provided, answer with "I don't know".

Context: {context}

Question: {question}""")

response = template.invoke(
    {
        "context": "The most recent advancements in NLP are being driven by Large Language Model (LLM). These models outperform their smaller counterparts and have become invaluable for developers who are creating applications with NLP capabilities. Developers can tap into these models through Hugging Face's 'transformers' library, or by utilizing OpenAI and cohere's offerings through the `openai` and `cohere` libraries, respectively.",
        "question": "Which model providers offers LLMs?"
    }
)
print(response)

text='Answer the questions based on the context below.\nIf the question cannot be answered using the information provided, answer with "I don\'t know".\n\nContext: The most recent advancements in NLP are being driven by Large Language Model (LLM). These models outperform their smaller counterparts and have become invaluable for developers who are creating applications with NLP capabilities. Developers can tap into these models through Hugging Face\'s \'transformers\' library, or by utilizing OpenAI and cohere\'s offerings through the `openai` and `cohere` libraries, respectively.\n\nQuestion: Which model providers offers LLMs?'


In [39]:
from langchain_core.prompts import PromptTemplate

template = PromptTemplate.from_template(
    """Responda a questão baseada no contexto a seguir. Se a questão não puder ser respondida usando a informação, responda usando 'Não sei a resposta para essa pergunta!'

Contexto: {contexto}

Pergunta: {pergunta}"""
)

# 2. Invoke corrigido - fora do template e com aspas fechadas
response = template.invoke({
    "contexto": "Quais os frameworks para a criação de IAs, chatbots, mais usados para construção de agentes de IA.",
    "pergunta": "Qual é o framework mais performático e mais simples de aprender?"
})

# 3. PromptValue não tem .content, usar .text ou apenas print(response)
print(response.text)

Responda a questão baseada no contexto a seguir. Se a questão não puder ser respondida usando a informação, responda usando 'Não sei a resposta para essa pergunta!'

Contexto: Quais os frameworks para a criação de IAs, chatbots, mais usados para construção de agentes de IA.

Pergunta: Qual é o framework mais performático e mais simples de aprender?


`Both template and model can be reused many times`

In [31]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import PromptTemplate

template = PromptTemplate.from_template("""Answer the question based on the context below. If the question cannot be answered using the information provided, answer with "I don't know".

Context: {context}

Question: {question}

Answer: """)

model = ChatOpenAI(model="gpt-3.5-turbo")
# `prompt` and `completion` are the results of using template and model once

prompt = template.invoke(
    {
        "context": "The most recent advancements in NLP are being driven by Large Language Models (LLMs). These models outperform their smaller counterparts and have become invaluable for developers who are creating applications with NLP capabilities. Developers can tap into these models through Hugging Face's `transformers` library, or by utilizing OpenAI and Cohere's offerings through the `openai` and `cohere` libraries, respectively.",
        "question": "Which model providers offer LLMs?",
    }
)

response = model.invoke(prompt)
print(response)

content='Hugging Face, OpenAI, and Cohere offer Large Language Models (LLMs).' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 132, 'total_tokens': 150, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BzRyPTzlGfPMaGQdcaqegGsWcTvLo', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--e6d0d13a-ff28-4384-b09e-5d30e07e2e85-0' usage_metadata={'input_tokens': 132, 'output_tokens': 18, 'total_tokens': 150, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [32]:
from langchain_core.prompts import PromptTemplate
from langchain_openai.chat_models import ChatOpenAI
template = PromptTemplate.from_template(
    """Responda a questão baseada no contexto a seguir. Se a questão não puder ser respondida usando a informação, responda usando 'Não sei a resposta para essa pergunta!'

Contexto: {contexto}

Pergunta: {pergunta}"""
)
model = ChatOpenAI(model='gpt-4.1')

# 2. Invoke corrigido - fora do template e com aspas fechadas
prompt = template.invoke({
    "contexto": "Quais os frameworks para a criação de IAs, chatbots, mais usados para construção de agentes de IA.",
    "pergunta": "Qual é o framework mais performático e mais simples de aprender?"
})

# 3. PromptValue não tem .content, usar .text ou apenas print(response)
response = model.invoke(prompt)
print(response)

content='Não sei a resposta para essa pergunta!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 85, 'total_tokens': 93, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_799e4ca3f1', 'id': 'chatcmpl-BzRyPYcRwFRjXEEnwe7DqZOQWMbi3', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--668e05b7-76bd-4e18-b404-b8b91833c0f4-0' usage_metadata={'input_tokens': 85, 'output_tokens': 8, 'total_tokens': 93, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


Building an AI chat application, the `ChatPromptTemplate` can be used instead to provide dynamic inputs based on the role of the chat message:

In [33]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            'Answer the question based on the context below. If the question cannot be answered using the information provided, answer with "I don\'t know".',
        ),
        ("human", "Context: {context}"),
        ("human", "Question: {question}"),
    ]
)

response = template.invoke(
    {
        "context": "The most recent advancements in NLP are being driven by Large Language Models (LLMs). These models outperform their smaller counterparts and have become invaluable for developers who are creating applications with NLP capabilities. Developers can tap into these models through Hugging Face's `transformers` library, or by utilizing OpenAI and Cohere's offerings through the `openai` and `cohere` libraries, respectively.",
        "question": "Which model providers offer LLMs?",
    }
)

print(response)

messages=[SystemMessage(content='Answer the question based on the context below. If the question cannot be answered using the information provided, answer with "I don\'t know".', additional_kwargs={}, response_metadata={}), HumanMessage(content="Context: The most recent advancements in NLP are being driven by Large Language Models (LLMs). These models outperform their smaller counterparts and have become invaluable for developers who are creating applications with NLP capabilities. Developers can tap into these models through Hugging Face's `transformers` library, or by utilizing OpenAI and Cohere's offerings through the `openai` and `cohere` libraries, respectively.", additional_kwargs={}, response_metadata={}), HumanMessage(content='Question: Which model providers offer LLMs?', additional_kwargs={}, response_metadata={})]


In [34]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# both `template` and `model` can be reused many times

template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            'Answer the question based on the context below. If the question cannot be answered using the information provided, answer with "I don\'t know".',
        ),
        ("human", "Context: {context}"),
        ("human", "Question: {question}"),
    ]
)

model = ChatOpenAI()

# `prompt` and `completion` are the results of using template and model once

prompt = template.invoke(
    {
        "context": "The most recent advancements in NLP are being driven by Large Language Models (LLMs). These models outperform their smaller counterparts and have become invaluable for developers who are creating applications with NLP capabilities. Developers can tap into these models through Hugging Face's `transformers` library, or by utilizing OpenAI and Cohere's offerings through the `openai` and `cohere` libraries, respectively.",
        "question": "Which model providers offer LLMs?",
    }
)

print(model.invoke(prompt))

content='OpenAI and Cohere offer Large Language Models (LLMs).' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 137, 'total_tokens': 150, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BzRyQBtK9vgTLnOqDe0ONBm2dqgS9', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--e849bb9e-d8a1-4700-9f78-025ea1f40d93-0' usage_metadata={'input_tokens': 137, 'output_tokens': 13, 'total_tokens': 150, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


1. Padrão de Prompts Dinâmicos - Sim, é um padrão estabelecido!
O código que você está vendo segue um padrão muito comum e recomendado no LangChain:
Por que este padrão é importante?
```python
template = ChatPromptTemplate.from_messages([
    ("system", "Instrução do sistema..."),
    ("human", "Entrada do usuário: {variavel}"),
])
```
Reutilização: O template pode ser usado múltiplas vezes com diferentes dados
* Separação de responsabilidades: O prompt (template) é separado da execução (model)
* Manutenibilidade: Fácil de modificar e testar prompts
* Escalabilidade: Pode ser usado em pipelines complexos

Quando usar:
* Definir o papel(role)/comportamento(behave) do modelo.
* Estabelecer regras e limitações
* Configurar o contexto geral da conversa
* Definir o tom e estilo de resposta.


```python
("system", "Você é um tutor de programação Python. Sempre explique conceitos de forma simples e forneça exemplos práticos.")
```

`HumanMessage` (Mensagem Humana)

```python
("human", "Pergunta: {question}")
```

Quando Usar:

* Entradas do usuário
* Perguntas específicas
* Dados que precisam ser processados
* Contexto que o usuário fornece

```python
("human", "Explique o conceito de list comprehension em Python")
```

`AIMessage (Mensagem da IA)`
```python
("ai", "Resposta da IA...")
```

Quando Usar:

* Simular conversar anteriores
* Fornecer exemplos de respostas
* Criar contextos de few-shot learning
* Manter histórico de conversas

Exemplo prático:
```python
("ai", "List comprehension é uma forma concisa de criar listas em Python. Exemplo: [x*2 for x in range(5)]")
```

`Estrutura Recomendada para Prompts Dinâmicos`
___
```python
# 1. Definir o template com tipos apropriados
template = ChatPromptTemplate.from_messages([
    ("system", "Defina o papel/contexto do modelo"),
    ("human", "Forneça o contexto: {context}"),
    ("human", "Faça a pergunta: {question}"),
])

# 2. Criar o modelo
model = ChatOpenAI()

# 3. Invocar com dados dinâmicos
prompt = template.invoke({
    "context": "seu contexto aqui",
    "question": "sua pergunta aqui"
})

# 4. Obter a resposta
response = model.invoke(prompt)
```

`**Boas Práticas para Desenvolvedores de Agentes**`
___

1.Sempre use templates para prompts dinâmicos

    -Evite strings hardcoded
    -Facilita testes e modificações

2.Separe claramente os tipos de mensagens

    -System: configuração/contexto
    -Human: entrada do usuário
    -AI: respostas/exemplos

3.Use variáveis para dados dinâmicos

    -{context}, {question}, {user_input}
    -Facilita a reutilização

4.Mantenha prompts modulares

    -Cada template com responsabilidade específica
    -Combine templates para casos complexos

**Versão básica**
```python
template = ChatPromptTemplate.from_messages([
    ("system", "Responda baseado no contexto."),
    ("human", "Contexto: {context}"),
    ("human", "Pergunta: {question}"),
])
```
**Versão avançada(para agentes)**
```python
template = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente especializado em análise de dados."),
    ("human", "Contexto: {context}"),
    ("human", "Pergunta: {question}"),
    ("ai", "Vou analisar o contexto e responder sua pergunta."),
    ("human", "Por favor, seja específico e forneça exemplos."),
])
```

Para lembrar:

 - `SystemMessage` = "Quem você é"
 - `HumanMessage` = "O que o usuário quer"
 - `AIMessage` = "Como você responde"

In [40]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

chat = ChatOpenAI(temperature=0)

messages = [
    SystemMessage(content="Você é um assistente prestativo, especialista em Matemática."),
    HumanMessage(content="Explique o que é Machine Learning em 50 palavras.")
]

response = chat.invoke(messages)
print(response.content)

Machine Learning é um campo da inteligência artificial que se concentra no desenvolvimento de algoritmos e modelos que permitem aos computadores aprender e melhorar a partir de dados. Esses modelos são treinados para fazer previsões, identificar padrões e tomar decisões sem serem explicitamente programados para cada tarefa.


`PromptTemplate` simples é só para texto, `ChatPromptTemplate` é para conversas com roles.

In [43]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate

chat = ChatOpenAI(model="gpt-3.5-turbo")

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("Você é um especialista em {tópico}"),
    HumanMessagePromptTemplate.from_template("Explique {assunto} em 35 palavras")
])

# Template
formatted_prompt = prompt.format_messages(
    tópico="framework",
    assunto="langchain"
    )

response = chat.invoke(formatted_prompt)
print(response.content)

Langchain é um framework de código aberto que facilita algoritmos e modelos de machine learning de linguagem natural. Ele fornece componentes pré-construídos e ferramentas para desenvolvedores criarem aplicações de processamento de linguagem.


## Getting Specific Formats out of LLMs
___

The most common format to generate with LLMs is JSON, which can be sent over the to your frontend code or be saved to a databased.

1. First task, define the output schema (from LLM). Then include that schema in the prompt , along with the text you want to use as the source.
2. In order to define a `schema`, this is easiest to do with `Pydantic`(*library* used for validating data against schemas)


In [45]:
from langchain_openai import ChatOpenAI
from pydantic import BaseModel

class AnswerWithJustification(BaseModel):
    """An answer to the user's question along with justification for the answer."""
    answer: str
    """The answer to the user's question"""
    justification: str
    """Justification for the answer"""


model = ChatOpenAI(model="gpt-4.1", temperature=0)
structured_model = model.with_structured_output(AnswerWithJustification)

response = structured_model.invoke(
    "What weighs more, a pound of bricks or a pound of feathers"
)
print(response)

answer='They weigh the same.' justification='A pound is a unit of weight. Therefore, a pound of bricks and a pound of feathers both weigh exactly one pound, regardless of the material. The difference is in their volume and density, not their weight.'


#### TUTORIAL: LangChain + Pandantic - Structured Output
========================================================

Este exemplo demonstra como usar Pydantic com LangChain para:
1. Validar dados de entrada e saída
2. Garantir formato estruturado das respostas
3. Tornar o código mais robusto e type-safe


In [46]:
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import Optional
import json
from datetime import datetime

In [47]:
class AnswerWithJustification(BaseModel):
    """
    Modelo que representa uma resposta estruturada com justificativa.

    💡 Por que usar Pydantic aqui?
    - Validação automática de tipos
    - Documentação clara dos campos
    - Serialização JSON automática
    - Integração nativa com LangChain
    """
    answer: str = Field(
        description="Resposta clara e concisa para a pergunta",
        min_length=1,
        max_length=500
    )

    justification: str = Field(
        description = "Explicação detalhada do raciocínio",
        min_length=10,
        max_length=100
    )

    confidence_level: Optional[float] = Field(
        default=None,
        description="Nível de confiança (0-1) na resposta",
        ge=0.0,
        le=1.0
    )

    timestamp: Optional[str] = Field(
        default_factory=lambda: datetime.now().isoformat(),
        description="Timestamp da resposta"
    )

    def display_response(self) ->str:
        """
        Método para exibir a resposta de forma mais legível.

        🔧 BENEFÍCIO: Evita o scroll infinito no output!
        """
        output = f"""
┌─ 💬 RESPOSTA ─────────────────────────────────────────┐
│ {self.answer}
├─ 🤔 JUSTIFICATIVA ───────────────────────────────────┤
│ {self.justification}
"""

        if self.confidence_level:
            confidence_bar = "█" * int(self.confidence_level * 10)
            output += f"""├─ 📊 CONFIANÇA ───────────────────────────────────────┤
│ {confidence_bar} {self.confidence_level:.1%}
"""

        output += f"""└─ 🕒 {self.timestamp} ──────────────────────────────────┘"""
        return output

#### 🚀 CLASSE PRINCIPAL PARA DEMONSTRAÇÃO

`class LangChainPydanticDemo`

In [51]:
class LangChainPydanticDemo:
    """
    Conceitos Importantes:
    1. Structured Output: LangChain força o LLM a retornar dados no formato exato do
    modelo Pydantic.

    2.Type safety: Pydantic valida automaticamente os tipos

    3.Error Handling: Falhas na validação são capturadas
    """
    def __init__(self, model_name = "gpt-4.1"):
        self.model = ChatOpenAI(
                model = model_name,
                temperature=0.1, #* baixa criatividade para respostas consistentes
        )

    #* 🔑 PONTO CHAVE: with_structured_output() força o formato
        self.structured_model = self.model.with_structured_output(
            AnswerWithJustification
        )

    def ask_question(self, question: str) -> AnswerWithJustification:
        """
        Faz uma pergunta e retorna resposta estruturada.

        🎯 BENEFÍCIOS do Structured Output:
        - Resposta sempre no formato esperado
        - Validação automática dos dados
        - Facilita integração com APIs e bancos de dados
        """
        try:
            response = self.structured_model.invoke(question)
            return response

        except Exception as e:
            #* Em caso de erro, retorna resposta padrão
            return AnswerWithJustification(
                answer="Erro ao processar pergunta",
                justification=f"Erro técnico: {str(e)}",
                confidence_level=0.0
            )

    def demo_multiple_questions(self):
        """
        Demonstra o uso com múltiplas perguntas para mostrar consistência.
        """
        questions = [
            "O que pesa mais: 1kg de chumbo ou 1kg de algodão?",
            "Por que o céu é azul?",
            "Qual a diferença entre Python e JavaScript?"
        ]

        print("🎓 DEMONSTRAÇÃO: Múltiplas perguntas com formato consistente\n")

        for i, question in enumerate(questions, 1):
            print(f"❓ PERGUNTA {i}: {question}")
            response = self.ask_question(question)
            print(response.display_response())
            print("\n" + "="*60 + "\n")


#### 📖 EXEMPLOS PRÁTICOS DE USO

In [52]:
def exemplo_basico():
    """Exemplo básico de uso do structured output."""
    print("🔵 EXEMPLO 1: Uso Básico")
    print("-" * 40)

    demo = LangChainPydanticDemo()

    question = "Explique o conceito de recursão em programação"
    response = demo.ask_question(question)

    # ✨ SAÍDA LIMPA - sem scroll infinito!
    print(response.display_response())

    # 💾 BONUS: Fácil conversão para JSON
    print("\n🔧 BONUS - Dados em JSON:")
    print(json.dumps(response.model_dump(), indent=2, ensure_ascii=False))

In [53]:
exemplo_basico()

🔵 EXEMPLO 1: Uso Básico
----------------------------------------

┌─ 💬 RESPOSTA ─────────────────────────────────────────┐
│ Recursão é um conceito em programação onde uma função chama a si mesma para resolver um problema, geralmente dividindo-o em subproblemas menores e semelhantes ao original.
├─ 🤔 JUSTIFICATIVA ───────────────────────────────────┤
│ A recursão é usada quando um problema pode ser decomposto em versões menores de si mesmo. Por meio d
├─ 📊 CONFIANÇA ───────────────────────────────────────┤
│ █████████ 98.0%
└─ 🕒 2024-06-19T18:00:00Z ──────────────────────────────────┘

🔧 BONUS - Dados em JSON:
{
  "answer": "Recursão é um conceito em programação onde uma função chama a si mesma para resolver um problema, geralmente dividindo-o em subproblemas menores e semelhantes ao original.",
  "justification": "A recursão é usada quando um problema pode ser decomposto em versões menores de si mesmo. Por meio d",
  "confidence_level": 0.98,
  "timestamp": "2024-06-19T18:00:00Z"
}


In [54]:
def exemplo_avancado():
    """Exemplo avançado mostrando validação."""
    print("\n🟢 EXEMPLO 2: Validação com Pydantic")
    print("-" * 40)

    # Tentativa de criar resposta inválida (para mostrar validação)
    try:
        invalid_response = AnswerWithJustification(
            answer="",  # ❌ Inválido: muito curto
            justification="Curto",  # ❌ Inválido: muito curto
            confidence_level=1.5  # ❌ Inválido: > 1.0
        )
    except Exception as e:
        print(f"❌ VALIDAÇÃO PYDANTIC FUNCIONOU: {e}")

    # ✅ Resposta válida
    valid_response = AnswerWithJustification(
        answer="Python é uma linguagem de programação",
        justification="Python é conhecida por sua sintaxe simples e legível",
        confidence_level=0.9
    )

    print("\n✅ RESPOSTA VÁLIDA:")
    print(valid_response.display_response())


In [55]:
exemplo_avancado()


🟢 EXEMPLO 2: Validação com Pydantic
----------------------------------------
❌ VALIDAÇÃO PYDANTIC FUNCIONOU: 3 validation errors for AnswerWithJustification
answer
  String should have at least 1 character [type=string_too_short, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/string_too_short
justification
  String should have at least 10 characters [type=string_too_short, input_value='Curto', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/string_too_short
confidence_level
  Input should be less than or equal to 1 [type=less_than_equal, input_value=1.5, input_type=float]
    For further information visit https://errors.pydantic.dev/2.11/v/less_than_equal

✅ RESPOSTA VÁLIDA:

┌─ 💬 RESPOSTA ─────────────────────────────────────────┐
│ Python é uma linguagem de programação
├─ 🤔 JUSTIFICATIVA ───────────────────────────────────┤
│ Python é conhecida por sua sintaxe simples e legível
├─ 📊 CONFIANÇA 

In [56]:
# Demonstração completa
print("\n🎓 DEMONSTRAÇÃO COMPLETA:")
print("="*60)
demo = LangChainPydanticDemo()
demo.demo_multiple_questions()


🎓 DEMONSTRAÇÃO COMPLETA:
🎓 DEMONSTRAÇÃO: Múltiplas perguntas com formato consistente

❓ PERGUNTA 1: O que pesa mais: 1kg de chumbo ou 1kg de algodão?

┌─ 💬 RESPOSTA ─────────────────────────────────────────┐
│ 1kg de chumbo e 1kg de algodão pesam exatamente o mesmo: 1kg cada.
├─ 🤔 JUSTIFICATIVA ───────────────────────────────────┤
│ A pergunta compara massas iguais de materiais diferentes. O peso é uma medida da força gravitacional
├─ 📊 CONFIANÇA ───────────────────────────────────────┤
│ ██████████ 100.0%
└─ 🕒 2023-06-19T12:00:00Z ──────────────────────────────────┘


❓ PERGUNTA 2: Por que o céu é azul?

┌─ 💬 RESPOSTA ─────────────────────────────────────────┐
│ O céu é azul porque a luz do Sol, ao passar pela atmosfera da Terra, sofre espalhamento pelas moléculas de ar, e a luz azul é espalhada em todas as direções com mais intensidade do que outras cores.
├─ 🤔 JUSTIFICATIVA ───────────────────────────────────┤
│ A luz branca do Sol é composta por várias cores, cada uma com um compr

🎓 LIÇÕES APRENDIDAS:

1. **Pydantic + LangChain = Dados Estruturados**
   - LLM retorna sempre no formato correto
   - Validação automática de tipos e valores

2. **Benefícios para Produção**
   - Code completion no IDE
   - Documentação automática
   - Integração fácil com APIs

3. **Melhor UX**
   - Saída formatada e legível
   - Sem scroll infinito
   - Informações organizadas

4. **Type Safety**
   - Erros capturados em desenvolvimento
   - Código mais robusto
   - Manutenção facilitada

**Por que Pydantic é Fundamental no LangChain**:

1. Structured Output: Força o LLM a retornar dados no formato exato
2. Validação Automática: Tipos e valores são verificados automaticamente
3. Integração Nativa: LangChain foi projetado para trabalhar com Pydantic
4. Produção Ready: Facilita APIs, banco de dados e frontend

#### Other Machine-Readable Formats with Output Parsers
___

We can also use an LLM or chat model to produce output in other formats, such as CSV or XML. Using output parsers  that are classes that help us structure large language model responses.

    - Providing format instructions
    - Validating and parsing output

In [57]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

parser = CommaSeparatedListOutputParser()

response = parser.invoke("apple, banana, cherry")
print(response)

['apple', 'banana', 'cherry']
